# Fitting a Hologram

This notebook shows how to load a pre-recorded hologram of a colloidal sphere
into a Feature object and analyze it with Lorenz-Mie theory. The analysis yields
the radius of the sphere, the refractive index of the sphere, and the sphere's
three-dimensional location relative to the center of the imaging plane.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
import cv2

from pylorenzmie.utilities import configuration as config
configuration.use_catch = False
from pylorenzmie.analysis import Frame

## Framework for Lorenz-Mie analysis
First, we instantiate a Frame() object. By default, Frame analyzes features with 
the standard Lorenz-Mie model for hologram formation. The model includes a description of the holographic microscope that records the images we wish to analyze. In this example, we specify that the instrument illuminates the sample with blue light at a vacuum wavelength of $\lambda$ = 0.447 μm and has a system magnification of 0.048 μm/pixel. For particles dispersed in water, the refractive index of the medium is $n_m$ = 1.340 at the imaging wavelength. These calibration constants may differ for different hardware implementations.
The commercial xSight instrument, for example, uses a system magnification of 0.120 μm/pixel.

In [2]:
configuration = dict(wavelength = 0.447,    # [um]
                     magnification = 0.048, # [um/pixel]
                     n_m = 1.34, 
                     distribution = 'radial',
                     percentpix = 0.2)

frame = Frame(**configuration)

Adding autoShape... 


## Read in holographic microscopy data
Next, we load the (normalized) hologram we wish to fit. In the present example, we use OpenCV to load a pre-recorded hologram that was saved as a PNG image. We convert the image to grayscale and normalize it by the mean value. This hologram was cropped from a larger image. Since we are not interested in locating the particle within the original image, we provide the model with a standard coordinate system using the coordinates() helper function. We finally can provide the normalized image data to the Feature() object for analysis.

In [3]:
def report(frame, results):
    fig, ax = plt.subplots(figsize=(8,8))
    ax.imshow(frame.data, cmap='gray')
    ax.scatter(results.x_p, results.y_p, c='r')
    for bbox in frame.bboxes:
        ax.add_patch(Rectangle(*bbox, fill=False, ec='r'))
    print(results)

In [4]:
img = cv2.imread('tutorials/image0400.png', 0).astype(float)
img /= np.mean(img)

## Load the recorded image into the Frame

In [5]:
results = frame.analyze(img)

(1024, 1280)


In [6]:
report(frame, results)

          x_p      dx_p         y_p      dy_p         z_p       dz_p  \
0  582.535988  0.033433  686.114768  0.034614  211.599038   0.186332   
1  883.871665  0.050109  747.508430  0.049321  167.031244   0.228911   
2  760.513669  6.215045  398.406706  5.625099  150.771592  18.432974   

        a_p      da_p       n_p      dn_p  success   npix    redchi  
0  1.160580  0.001870  1.391965  0.000282     True  27899  3.868764  
1  0.934735  0.002534  1.380916  0.000351     True  30574  3.544586  
2  0.480071  0.263653  1.340549  0.000778     True  71865  1.810697  


## Now that it's all set up, run it again

In [7]:
img = cv2.imread('tutorials/image0010.png', 0).astype(float)
img /= np.mean(img)
results = frame.analyze(img)
report(frame, results)

(1024, 1280)
           x_p        dx_p         y_p        dy_p         z_p        dz_p  \
0   562.217207    0.044233  720.628483    0.040692  177.134952    0.160627   
1   920.234929    0.044626  732.413408    0.040702  177.471345    0.161296   
2   753.824879  208.873562  395.240576  190.218669  151.635065  606.510864   
3  1107.840097    1.688901  200.585400    1.501533  134.561731    4.804376   

        a_p      da_p       n_p      dn_p  success   npix    redchi  
0  2.088812  0.001684  1.369545  0.000089     True  58861  6.806455  
1  2.060395  0.001686  1.369395  0.000088     True  60169  6.551660  
2  0.413834  8.805754  1.340025  0.001359     True  70092  2.868680  
3  0.292402  0.082548  1.336239  0.002533     True  27829  1.611335  
